In [ ]:
# inject high intensity signals to verify that we can recover them after they go through SM-Emp 
# in the shifted spectra and the residuals.
# Then should check that they are recovered by the search algorithm, of course.

import numpy as np
import os
import matplotlib.pyplot as plt
from astropy.io import fits
import pandas as pd
import math
from scipy.interpolate import interp1d
from mpmath import mp
mp.dps=100
exp_array = np.frompyfunc(mp.exp, 1, 1)

In [ ]:
# make a directory for the injected spectra -- only run this once
#if not os.path.isdir('/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/injected_spectra'):
#    os.mkdir('/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/injected_spectra')

In [ ]:
# function to insert simulated gaussians
def insert_gaussian(spectrum, gaussian_params, midpoint, numpoints):
    height = gaussian_params[0]
    position = gaussian_params[1] # position within segment, not index in spectrum
    FWHM = gaussian_params[2]
    offset = gaussian_params[3]
    x = np.linspace(0,numpoints-1,numpoints) # numpoints must be odd
    gauss = gaussian(x,height,position,FWHM/(2*np.sqrt(2*np.log(2))),offset)
    new_spect = spectrum
    new_spect[midpoint - math.floor(numpoints/2):midpoint + math.floor(numpoints/2)] = spectrum[midpoint - math.floor(numpoints/2):midpoint + math.floor(numpoints/2)] + gauss
    return new_spect
    
def gaussian(x,a,b,c,d): # a = height, b = position of peak, c = width, x = numpy array of x values
    f = a*exp_array((-(x-b)**2)/(2*c)**2) + d
    return f 

In [22]:
test_stars = ['GJ1002', # -> cool (T ~ 3100K)
              'HIP115562', # -> relatively cool (T ~ 3700K)
              'GJ144', # -> medium temperature (T ~ 5000K)
              'HIP13834'] # -> hot (T ~ 6300K)

star_list = os.listdir('APF_spectra/all_apf_spectra_highest_SNR')[0:300]

heights = [1.5, 2, 3] # times the 98th percentile of order 40


#if not os.path.isdir('APF_spectra/injected_spectra_' + str(height)):
#   os.mkdir('APF_spectra/injected_spectra_' + str(height))
apf_wave = fits.open('apf_wav.fits')
j = 0
injected_peak_photons = np.zeros(len(star_list)*3) # *3 becuase three different inejctions in each star
median_photons_100_pix = np.zeros(len(star_list)*3)
injection_wl = np.zeros(len(star_list)*3)*np.nan
stars = []
if not os.path.isdir('APF_spectra/injected_spectra'):
    os.mkdir('APF_spectra/injected_spectra')
    
for star_name in star_list:
    star = star_name.split('_')[0]
    for m in range(3): stars += [star] # want three entries for this star: one for each injection
    dir_path = 'APF_spectra/all_apf_spectra_highest_SNR/' + star + '_spectra'
    i = 0
    print('Star: ' + star)
    for file in os.listdir(dir_path):
        hdu = fits.open(dir_path + '/' + file)# , mode='update')
        spectrum = np.copy(hdu[0].data)
        color = 'C' + str(i)  
        n = 0
        orders = [39, 40, 41]
        y = hdu[0].data
        for height in heights:
            order = orders[n] # inject the 1.5 hieght into order 39, 2 into order 40, etc
            y_order = y[order]
            h = height - 1 # becuase in the above implementation we add to the baseline, so ie. h = 1 injects a guassian whose peak is at twice the pixels value of the 98th percentile
            print('    file ' + file + ': injecting signal of W = 5, H = ' + str(height) + 
                  '*(98th pct level of order) = ' + str(np.round(height * np.percentile(y_order,98),3)) + ' into order 40.')  
            median_100 = np.median(y_order[int(len(y_order)/2 - 50): int(len(y_order)/2 + 50)])
            y_new = insert_gaussian(y_order, [height*np.percentile(y_order,98), 49, 20, 0], int(len(y_order)/2), 100)
            injection_center_wl = apf_wave[0].data[order][int(len(y_order)/2)]
            print('                     : injecting at center point ' + str(injection_center_wl) + ' A')
            #y[int(len(y)/2)] = 10 * np.percentile(y,99)
            #x = apf_wave[0].data[order]
            #plt.plot(x, y[:-1], '.', c = color)
            #plt.xlim([5553,5557])
            injected_peak_photons[3*j + n] = injected_peak_photons[3*j + n] + height * np.percentile(y_order,98) # sum over each file
            median_photons_100_pix[3*j + n] = median_photons_100_pix[3*j + n] + median_100 # sum over each file
            injection_wl[3*j + n] = injection_center_wl # same for each file (and each star)
            spectrum[order,:] = y_new
            n += 1
        #plt.figure(star + ', ' + file)
        #for order in np.arange(38,43,1):
        #    plt.plot(apf_wave[0].data[order], spectrum[order,:-1])
        #    plt.title(star + ', ' + file)
        i += 1

        new_header = hdu[0].header # get header
        new_header.set('Injected', 'YES','Spectrum contains injected gaussian for testing.')
        data_hdu = fits.PrimaryHDU(spectrum, new_header) 
        hdu_new = fits.HDUList([data_hdu])
        out_path = 'APF_spectra/injected_spectra/'
        if not os.path.isdir(out_path + star + '_spectra/'):
            os.mkdir(out_path + star + '_spectra/')
        #hdu_new.writeto(out_path + star + '_spectra/' + file + '_injected' + '.fits')
        hdu.close()
        
    j += 1

save_injection_key = False
if save_injection_key:
    save_data = list(zip(stars, injection_wl, injected_peak_photons, median_photons_100_pix))
    df = pd.DataFrame(save_data,
              columns = ['Star_name', 'Injection_wavelength', 'Injected_peak_photons', 'Local_median_photons'])
    if os.path.exists('Injection_key_full.csv'):
        df.to_csv('Injection_key_full.csv', mode='a', index = False, header=False)
    else:
        df.to_csv('Injection_key_full.csv', index = False)

Star: HIP109980
    file rbob.394.fits: injecting signal of W = 5, H = 1.5*(98th pct level of order) = 4880.957 into order 40.
                     : injecting at center point 5490.07413621209 A
    file rbob.394.fits: injecting signal of W = 5, H = 2*(98th pct level of order) = 6809.051 into order 40.
                     : injecting at center point 5555.36527738973 A
    file rbob.394.fits: injecting signal of W = 5, H = 3*(98th pct level of order) = 10311.392 into order 40.
                     : injecting at center point 5622.229264717167 A
    file rbob.396.fits: injecting signal of W = 5, H = 1.5*(98th pct level of order) = 4800.042 into order 40.
                     : injecting at center point 5490.07413621209 A
    file rbob.396.fits: injecting signal of W = 5, H = 2*(98th pct level of order) = 6709.871 into order 40.
                     : injecting at center point 5555.36527738973 A
    file rbob.396.fits: injecting signal of W = 5, H = 3*(98th pct level of order) = 10143.86

    file rbob.395.fits: injecting signal of W = 5, H = 1.5*(98th pct level of order) = 4800.015 into order 40.
                     : injecting at center point 5490.07413621209 A
    file rbob.395.fits: injecting signal of W = 5, H = 2*(98th pct level of order) = 6735.552 into order 40.
                     : injecting at center point 5555.36527738973 A
    file rbob.395.fits: injecting signal of W = 5, H = 3*(98th pct level of order) = 10187.105 into order 40.
                     : injecting at center point 5622.229264717167 A
Star: HIP11000
    file ratn.221.fits: injecting signal of W = 5, H = 1.5*(98th pct level of order) = 11454.764 into order 40.
                     : injecting at center point 5490.07413621209 A
    file ratn.221.fits: injecting signal of W = 5, H = 2*(98th pct level of order) = 15926.943 into order 40.
                     : injecting at center point 5555.36527738973 A
    file ratn.221.fits: injecting signal of W = 5, H = 3*(98th pct level of order) = 24145.1

In [20]:
# how to inject into somewhere other than the peak of the blaze function 
star_list = os.listdir('APF_spectra/all_apf_spectra_highest_SNR')[300:600]

heights = [1.5, 2, 3] # times the 98th percentile of order 40


#if not os.path.isdir('APF_spectra/injected_spectra_' + str(height)):
#   os.mkdir('APF_spectra/injected_spectra_' + str(height))
apf_wave = fits.open('apf_wav.fits')
j = 0
injected_peak_photons = np.zeros(len(star_list)*3) # *3 becuase three different inejctions in each star
median_photons_100_pix = np.zeros(len(star_list)*3)
injection_wl = np.zeros(len(star_list)*3)*np.nan
stars = []
if not os.path.isdir('APF_spectra/injected_spectra'):
    os.mkdir('APF_spectra/injected_spectra')
    
for star_name in star_list:
    star = star_name.split('_')[0]
    for m in range(3): stars += [star] # want three entries for this star: one for each injection
    dir_path = 'APF_spectra/all_apf_spectra_highest_SNR/' + star + '_spectra'
    i = 0
    print('Star: ' + star)
    for file in os.listdir(dir_path):
        hdu = fits.open(dir_path + '/' + file)# , mode='update')
        spectrum = np.copy(hdu[0].data)
        color = 'C' + str(i)  
        n = 0
        orders = [39, 40, 41]
        y = hdu[0].data
        for height in heights:
            # Inject into the first of the overlapping orders
            order = orders[n] # inject the 1.5 hieght into order 39, 2 into order 40, etc
            frac = 5
            y_order = y[order]
            injection_center_wl = apf_wave[0].data[order][int(len(y_order)/frac)] # deliberately inject into the order edge (overlapping region)
            h = height - 1 # becuase in the above implementation we add to the baseline, so ie. h = 1 injects a guassian whose peak is at twice the pixels value of the 98th percentile  
            # is this close to ok near edges??
            median_100 = np.median(y_order[int(len(y_order)/frac - 50): int(len(y_order)/frac + 50)])
            second_order = order - 1
            y_second_order = y[second_order] 
            second_median_100 = np.median(y_second_order[int(len(y_second_order)/frac - 50): int(len(y_order)/frac + 50)]) 
            combined_median_100 = median_100 + second_median_100
            num_photons = height * combined_median_100
            y_new = insert_gaussian(y_order, [num_photons, 49, 20, 0], int(len(y_order)/frac), 100)
            print('    file ' + file + ': injecting signal of W = 5, H = ' + str(height) + 
                  '*(local median) = ' + str(np.round(num_photons,3)) + '. ')
            print('                     : injecting at center point ' + str(injection_center_wl) + ' A, order ' + str(order))
            
            # Inject into the second of the overlapping orders

            #second_y_new = insert_gaussian(y_second_order, [height*np.percentile(y_second_order,98), 49, 20, 0], int(len(y_second_order)/frac), 100)
            #print('                     : injecting at center point ' + str(injection_center_wl) + ' A ' + str(second_order))
            
            #y[int(len(y)/2)] = 10 * np.percentile(y,99)
            #x = apf_wave[0].data[order]
            #plt.plot(x, y[:-1], '.', c = color)
            #plt.xlim([5553,5557])
            injected_peak_photons[3*j + n] = injected_peak_photons[3*j + n] + height * combined_median_100 # sum over each file
            median_photons_100_pix[3*j + n] = median_photons_100_pix[3*j + n] + combined_median_100 # sum over each file
            injection_wl[3*j + n] = injection_center_wl # same for each file (and each star)
            spectrum[order,:] = y_new
            n += 1
        #plt.figure()
        #for order in np.arange(38,43,1):
        #    plt.plot(apf_wave[0].data[order], spectrum[order,:-1])
        #    plt.title(star + ', ' + file)
        i += 1

        new_header = hdu[0].header # get header
        new_header.set('Injected', 'YES','Spectrum contains injected gaussian for testing.')
        data_hdu = fits.PrimaryHDU(spectrum, new_header) 
        hdu_new = fits.HDUList([data_hdu])
        out_path = 'APF_spectra/injected_spectra_order_edges/'
        if not os.path.isdir(out_path + star + '_spectra/'):
            os.mkdir(out_path + star + '_spectra/')
        #hdu_new.writeto(out_path + star + '_spectra/' + file + '_injected' + '.fits')
        hdu.close()
        
    j += 1

save_injection_key = True
if save_injection_key:
    save_data = list(zip(stars, injection_wl, injected_peak_photons, median_photons_100_pix))
    df = pd.DataFrame(save_data,
              columns = ['Star_name', 'Injection_wavelength', 'Injected_peak_photons', 'Local_median_photons'])
    if os.path.exists('Injection_key_full.csv'):
        df.to_csv('Injection_key_full.csv', mode = 'a', index = False, header = False)
    else:
        df.to_csv('Injection_key_full.csv', index = False)

Star: HIP25878
    file rbnp.951.fits: injecting signal of W = 5, H = 1.5*(local median) = 10227.092. 
                     : injecting at center point 5461.813824900171 A, order 39
    file rbnp.951.fits: injecting signal of W = 5, H = 2*(local median) = 18960.688. 
                     : injecting at center point 5526.772619756325 A, order 40
    file rbnp.951.fits: injecting signal of W = 5, H = 3*(local median) = 35803.852. 
                     : injecting at center point 5593.298646595918 A, order 41
    file rbnp.952.fits: injecting signal of W = 5, H = 1.5*(local median) = 11148.643. 
                     : injecting at center point 5461.813824900171 A, order 39
    file rbnp.952.fits: injecting signal of W = 5, H = 2*(local median) = 20782.051. 
                     : injecting at center point 5526.772619756325 A, order 40
    file rbnp.952.fits: injecting signal of W = 5, H = 3*(local median) = 38926.515. 
                     : injecting at center point 5593.298646595918 A, 